In [4]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install bs4
!{sys.executable} -m pip install datetime
!{sys.executable} -m pip install selenium
!{sys.executable} -m pip install nltk
!{sys.executable} -m pip install textblob
!{sys.executable} -m pip install lxml
!{sys.executable} -m pip install plotly

  Using cached https://files.pythonhosted.org/packages/10/ed/7e8b97591f6f456174139ec089c769f89a94a1a4025fe967691de971f314/bs4-0.0.1.tar.gz
  Stored in directory: C:\Users\Viraj\AppData\Local\pip\Cache\wheels\a0\b0\b2\4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
  Stored in directory: C:\Users\Viraj\AppData\Local\pip\Cache\wheels\d7\a9\33\acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying


Defaulting to user installation because normal site-packages is not writeable


In [20]:
# """
# This code was written by Viraj Shetty and Arjun Praveen!
# """
# import pandas as pd
# import time
# import operator
# import bs4 as bs
# import datetime
# from bs4 import BeautifulSoup
# import urllib.request
# from urllib.request import urlopen
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.firefox.options import Options
# from datetime import date
# from textblob import TextBlob

# """
# This little piece of code is added to make the webdriver run silently
# Generally when you use WebDriver in Selenium it opens a new window
# If you have automated some functions you can also see those functions happening live
# To avoid it use this option called "headless" which does all this in the background

# Web Driver object creation
# Note: Before importing Selenium Webdriver
#     Make sure you have a webdriver application installed in the same directory
    
#     For Mozzilla
#     >https://github.com/mozilla/geckodriver/releases
    
#     For Chrome
#     >https://chromedriver.chromium.org/downloads
# """
# options = Options()
# options.headless = True
# browser = webdriver.Firefox(options = options)


# # This function tokenizes the entered keywords

# def tokenizer(l_t, l_d):
#     v = TextBlob(l_t)
#     w = TextBlob(l_d)
#     a = v.noun_phrases
#     b = w.noun_phrases
#     c = a + b
#     d = ""
#     for i in c:
#         d = d + i + " "
#     print(d)
#     return d

# # Function for extracting number of days since the video has been uploaded

# def number_of_days(v_date):
#     today = date.today()
#     month = {
#              "Jan" : 1,
#              "Feb" : 2,
#              "Mar" : 3,
#              "Apr" : 4,
#              "May" : 5,
#              "Jun" : 6,
#              "Jul" : 7,
#              "Aug" : 8,
#              "Sep" : 9,
#              "Oct" : 10,
#              "Nov" : 11,
#              "Dec" : 12
#             }
#     v=v_date.split(" ")
#     v_year = int(v[2])
#     v_month = int(month[v[0]])
#     v_day = int(v[1])
#     v_date = date(v_year,v_month,v_day)
#     time = abs(today - v_date)
#     return time.days

# def search():
    
#     keyword = input("Enter title: ")
    
#     description = input("Enter description: ")  
    
#     keysearch = tokenizer(keyword, description)
#     keysearch_list = keysearch.split(" ")
#     key = []
#     for j in keysearch_list:
#         if j not in key:
#             key.append(j)
#     print(key)
#     keysearch = ""
#     for k in key:
#         keysearch = keysearch + k + " "
#     print("Scraping search results for the keyword: " + keysearch)
    
#     tag_list = keysearch.split(" ")
#     browser = webdriver.Firefox()
#     browser.get(f'https://www.youtube.com/results?search_query={keysearch}&sp=EgIQAQ%253D%253D')
#     browser.maximize_window()

#     """
#     Please note here you need to use an Adblock to avoid advertisement videos to be a part of your list
#     Add the filepath to your .xpi file
#     """
#     extension = "{your }" 
#     #browser.install_addon(extension, temporary = True)

#     # Change value of While Loop to scroll more pages
#     i=0
#     while i<=1:
#         html = browser.find_element_by_tag_name('html')
#         '''
#         The code in the next line takes the page to end. Since in YouTube if you want more
#         videos to load, you have to scroll down to the end of the page
#         '''
#         html.send_keys(Keys.END)
#         time.sleep(3)
#         i+=1
        
#     v_l = browser.find_elements_by_xpath('//*[@id="video-title"]')
#     link = []
#     for i in v_l:
#         link.append(i.get_attribute('href'))
        
#     #print(link) 
#     wait = WebDriverWait(browser,15)
#     df = pd.DataFrame(columns = ['Link','Title','URL','Views','Likes','Dislikes','Subcount','Date','Days','Score','Tags'])
#     k=0
#     for x in link:
#         try:
#             i = 0
#             browser.get(x)
#             time.sleep(3)
#             v_link = x
#             v_title = wait.until(EC.presence_of_element_located((By.XPATH,'//h1[@class="title style-scope ytd-video-primary-info-renderer"]'))).text
#             v_views = wait.until(EC.presence_of_element_located((By.XPATH,'/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[4]/div[1]/div/div[5]/div[2]/ytd-video-primary-info-renderer/div/div/div[1]/div[1]/yt-view-count-renderer/span[1]'))).text
#             try:    
#                 v_views = int(v_views.strip(' views').replace(',', '', 2))
#             except:
#                 v_views = 1
#             v_likes = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'ytd-toggle-button-renderer.ytd-menu-renderer:nth-child(1) > a:nth-child(1) > yt-formatted-string:nth-child(2)'))).text
#             v_dislikes = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'ytd-toggle-button-renderer.style-scope:nth-child(2) > a:nth-child(1) > yt-formatted-string:nth-child(2)'))).text
            
#             #if (v_likes == 0 & v_dislikes == 0):
#                 #v_like_dislike = 0
            
#             #Likes
#             try:
#                 if (operator.contains(v_likes,'K')):
#                     if(operator.contains(v_likes,'.')):
#                         pos = len(v_likes)-v_likes.find('.')-1 #length from the end
#                         a1 = v_likes.replace('.', '', 1)
#                         num = (int(a1[:len(a1)-1])*1000)/(10**(pos-1))
#                         v_likes = num
#                     else:
#                         v_likes=int(v_likes[ :len(v_likes)-1])*(1000)
#                 elif operator.contains(v_likes,'M'):
#                     if(operator.contains(v_likes,'.')):
#                         pos = len(v_likes)-v_likes.find('.')-1 #length from the end
#                         a1 = v_likes.replace('.', '', 1)
#                         num = (int(a1[:len(a1)-1])*1000000)/(10**(pos-1))
#                         v_likes = num
#                     else:
#                         v_likes=int(v_likes[ :len(v_likes)-1])*(1000000)
#                 else:
#                     v_likes = int(v_likes)
#             except:
#                 v_likes = 1

#             #Dislikes
#             try:
#                 if (operator.contains(v_dislikes,'K')):
#                     if(operator.contains(v_dislikes,'.')):
#                         pos = len(v_dislikes)-v_dislikes.find('.')-1 #length from the end
#                         a1 = v_dislikes.replace('.', '', 1)
#                         num = (int(a1[:len(a1)-1])*1000)/(10**(pos-1))
#                         v_dislikes = num
#                     else:
#                         v_dislikes=int(v_dislikes[ :len(v_dislikes)-1])*(1000)
#                 elif operator.contains(v_dislikes,'M'):
#                     if(operator.contains(v_dislikes,'.')):
#                         pos = len(v_dislikes)-v_dislikes.find('.')-1 #length from the end
#                         a1 = v_dislikes.replace('.', '', 1)
#                         num = (int(a1[:len(a1)-1])*1000000)/(10**(pos-1))
#                         v_dislikes = num
#                     else:
#                         v_dislikes=int(v_dislikes[ :len(v_dislikes)-1])*(1000000)
#                 else:
#                     v_dislikes = int(v_dislikes)
#             except:
#                 v_dislikes = 1
            
#             v_like_dislike = ((v_likes-v_dislikes)/v_likes)
            
#             v_subcount = wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="owner-sub-count"]'))).text
#             v_subcount = v_subcount.strip(" subscribers")
#             try:
#                 if operator.contains(v_subcount,'M'):
#                     if(operator.contains(v_subcount,'.')):
#                         pos = len(v_subcount)-v_subcount.find('.')-1 #length from the end
#                         a1 = v_subcount.replace('.', '', 1)
#                         num = (int(a1[:len(a1)-1])*1000000)/(10**(pos-1))
#                         v_subcount = num
#                     else:
#                         v_subcount=int(v_subcount[ :len(v_subcount)-1])*(1000000)
#                 elif operator.contains(v_subcount,'K'):
#                     if(operator.contains(v_subcount,'.')):
#                         pos = len(v_subcount)-v_subcount.find('.')-1 #length from the end
#                         a1 = v_subcount.replace('.', '', 1)
#                         num = (int(a1[:len(a1)-1])*1000)/(10**(pos-1))
#                         v_subcount = num
#                     else:
#                         v_subcount = int(v_subcount[ :len(v_subcount)-1])*(1000)
#                 else:
#                     v_subcount = int(float(v_subcount))
#             except:
#                 v_subcount = 1
#             v_urls = browser.find_elements_by_xpath('//div[@class="ytp-cued-thumbnail-overlay-image"]')
#             for i in v_urls:
#                 v_url=i.get_attribute('style')[23:-3]
#                 break
#             v_date = browser.find_element_by_xpath("//div[@id='date']").text
#             if(operator.contains(v_date,'Premiered')):
#                 v_date = v_date[10:]
#             elif(operator.contains(v_date,'Streamed live on')):
#                 v_date = v_date[17:]

#             v_date = v_date[1:].replace(",","",1)
#             v_days = number_of_days(v_date)
        
# #             try:
# #                 url = x
# #                 html_content = urlopen(url)
# #                 soup = BeautifulSoup(html_content,'lxml')
# #                 v_tags = [tag["content"] for tag in soup.findAll("meta",property="og:video:tag")]
# #             except:
# #                 print('Error')
# #             co = 0
            
# #             for tl in tag_list:
# #                 for t in v_tags:
# #                     if operator.contains(t, tl):
# #                         co+=1
# #             v_tag = co/(len(tag_list))
            
#             # Score formula
#             try:
#                 v_score = (((70*v_views)/700000) + (50*(v_likes/19000)) - (20*(v_dislikes/370)) + (160*v_like_dislike) +(20*(v_views/v_days)/7000)+(30*(v_likes/v_days)/200) + (12 * v_tag))/43
#             except:
#                 v_days = 1
#                 v_likes = 1
#                 v_score = (((70*v_views)/700000) + (50*(v_likes/19000)) - (20*(v_dislikes/370)) + (160*v_like_dislike)+(20*(v_views/v_days)/7000)+(30*(v_likes/v_days)/200) + (12 * v_tag))/43
            
            
#             print('{}: {}/{} ,{} , {:.2f}'.format(url,co,len(tag_list),v_tag,v_score))

            
#             df.loc[len(df)] = [v_link,v_title,v_url,v_views,v_likes,v_dislikes,v_subcount,v_date,v_days,v_score,v_tags]
#             k
#             k+=1
#             print(k)
#         #Exception added since some YouTube ads bypass the adblock
#         except Exception as e:
#             print(e)
#             continue
#     df = df.sort_values(by='Score', ascending = 0)
#     return df
#     browser.close()

# n = int(input("How many Links would you like to print? "))
# df1 = search()
# df = df1.nlargest(n,'Score')
# print(df)

How many Links would you like to print? 5
Enter title: Python
Enter description: 
python 
['python', '']
Scraping search results for the keyword: python  
Message: invalid type: null, expected a string at line 1 column 12

'Started'
name 'v_tag' is not defined
name 'v_tag' is not defined
name 'v_tag' is not defined


TypeError: Column 'Score' has dtype object, cannot use method 'nlargest' with this dtype

In [2]:
"""
This code was written by Viraj Shetty and Arjun Praveen!
"""
import pandas as pd
import time
import operator
import bs4 as bs
import datetime
from bs4 import BeautifulSoup
import urllib.request
from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from datetime import date
from textblob import TextBlob

"""
This little piece of code is added to make the webdriver run silently
Generally when you use WebDriver in Selenium it opens a new window
If you have automated some functions you can also see those functions happening live
To avoid it use this option called "headless" which does all this in the background

Web Driver object creation
Note: Before importing Selenium Webdriver
    Make sure you have a webdriver application installed in the same directory
    
    For Mozzilla
    >https://github.com/mozilla/geckodriver/releases
    
    For Chrome
    >https://chromedriver.chromium.org/downloads
"""
options = Options()
options.headless = True
browser = webdriver.Firefox(options = options)


# This function tokenizes the entered keywords

def tokenizer(l_t, l_d):
    v = TextBlob(l_t)
    w = TextBlob(l_d)
    a = v.noun_phrases
    b = w.noun_phrases
    c = a + b
    d = ""
    for i in c:
        d = d + i + " "
    print(d)
    return d

# Function for extracting number of days since the video has been uploaded

def number_of_days(v_date):
    today = date.today()
    month = {
             "Jan" : 1,
             "Feb" : 2,
             "Mar" : 3,
             "Apr" : 4,
             "May" : 5,
             "Jun" : 6,
             "Jul" : 7,
             "Aug" : 8,
             "Sep" : 9,
             "Oct" : 10,
             "Nov" : 11,
             "Dec" : 12
            }
    v=v_date.split(" ")
    v_year = int(v[2])
    v_month = int(month[v[0]])
    v_day = int(v[1])
    v_date = date(v_year,v_month,v_day)
    time = abs(today - v_date)
    return time.days

def search():
    
    keyword = input("Enter title: ")
    
    description = input("Enter description: ")  
    
    keysearch = tokenizer(keyword, description)
    keysearch_list = keysearch.split(" ")
    key = []
    for j in keysearch_list:
        if j not in key:
            key.append(j)
    print(key)
    keysearch = ""
    for k in key:
        keysearch = keysearch + k + " "
    print("Scraping search results for the keyword: " + keysearch)
    
    tag_list = keysearch.split(" ")
    browser = webdriver.Firefox()
    browser.get(f'https://www.youtube.com/results?search_query={keysearch}&sp=EgIQAQ%253D%253D')
    browser.maximize_window()

    """
    Please note here you need to use an Adblock to avoid advertisement videos to be a part of your list
    Add the filepath to your .xpi file
    """
    extension = "{your }" 
    #browser.install_addon(extension, temporary = True)

    # Change value of While Loop to scroll more pages
    i=0
    while i<=1:
        html = browser.find_element_by_tag_name('html')
        '''
        The code in the next line takes the page to end. Since in YouTube if you want more
        videos to load, you have to scroll down to the end of the page
        '''
        html.send_keys(Keys.END)
        time.sleep(3)
        i+=1
        
    v_l = browser.find_elements_by_xpath('//*[@id="video-title"]')
    link = []
    for i in v_l:
        link.append(i.get_attribute('href'))
        
    #print(link) 
    wait = WebDriverWait(browser,15)
    df = pd.DataFrame(columns = ['Link','Title','URL','Views','Likes','Dislikes','Subcount','Date','Days','Score'])
    k=0
    for x in link:
        try:
            i = 0
            browser.get(x)
            time.sleep(3)
            v_link = x
            v_title = wait.until(EC.presence_of_element_located((By.XPATH,'//h1[@class="title style-scope ytd-video-primary-info-renderer"]'))).text
            v_views = wait.until(EC.presence_of_element_located((By.XPATH,'/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[4]/div[1]/div/div[5]/div[2]/ytd-video-primary-info-renderer/div/div/div[1]/div[1]/yt-view-count-renderer/span[1]'))).text
            try:    
                v_views = int(v_views.strip(' views').replace(',', '', 2))
            except:
                v_views = 1
            v_likes = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'ytd-toggle-button-renderer.ytd-menu-renderer:nth-child(1) > a:nth-child(1) > yt-formatted-string:nth-child(2)'))).text
            v_dislikes = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'ytd-toggle-button-renderer.style-scope:nth-child(2) > a:nth-child(1) > yt-formatted-string:nth-child(2)'))).text
            
            #if (v_likes == 0 & v_dislikes == 0):
                #v_like_dislike = 0
            
            #Likes
            try:
                if (operator.contains(v_likes,'K')):
                    if(operator.contains(v_likes,'.')):
                        pos = len(v_likes)-v_likes.find('.')-1 #length from the end
                        a1 = v_likes.replace('.', '', 1)
                        num = (int(a1[:len(a1)-1])*1000)/(10**(pos-1))
                        v_likes = num
                    else:
                        v_likes=int(v_likes[ :len(v_likes)-1])*(1000)
                elif operator.contains(v_likes,'M'):
                    if(operator.contains(v_likes,'.')):
                        pos = len(v_likes)-v_likes.find('.')-1 #length from the end
                        a1 = v_likes.replace('.', '', 1)
                        num = (int(a1[:len(a1)-1])*1000000)/(10**(pos-1))
                        v_likes = num
                    else:
                        v_likes=int(v_likes[ :len(v_likes)-1])*(1000000)
                else:
                    v_likes = int(v_likes)
            except:
                v_likes = 1

            #Dislikes
            try:
                if (operator.contains(v_dislikes,'K')):
                    if(operator.contains(v_dislikes,'.')):
                        pos = len(v_dislikes)-v_dislikes.find('.')-1 #length from the end
                        a1 = v_dislikes.replace('.', '', 1)
                        num = (int(a1[:len(a1)-1])*1000)/(10**(pos-1))
                        v_dislikes = num
                    else:
                        v_dislikes=int(v_dislikes[ :len(v_dislikes)-1])*(1000)
                elif operator.contains(v_dislikes,'M'):
                    if(operator.contains(v_dislikes,'.')):
                        pos = len(v_dislikes)-v_dislikes.find('.')-1 #length from the end
                        a1 = v_dislikes.replace('.', '', 1)
                        num = (int(a1[:len(a1)-1])*1000000)/(10**(pos-1))
                        v_dislikes = num
                    else:
                        v_dislikes=int(v_dislikes[ :len(v_dislikes)-1])*(1000000)
                else:
                    v_dislikes = int(v_dislikes)
            except:
                v_dislikes = 1
            
            v_like_dislike = ((v_likes-v_dislikes)/v_likes)
            
            v_subcount = wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="owner-sub-count"]'))).text
            v_subcount = v_subcount.strip(" subscribers")
            try:
                if operator.contains(v_subcount,'M'):
                    if(operator.contains(v_subcount,'.')):
                        pos = len(v_subcount)-v_subcount.find('.')-1 #length from the end
                        a1 = v_subcount.replace('.', '', 1)
                        num = (int(a1[:len(a1)-1])*1000000)/(10**(pos-1))
                        v_subcount = num
                    else:
                        v_subcount=int(v_subcount[ :len(v_subcount)-1])*(1000000)
                elif operator.contains(v_subcount,'K'):
                    if(operator.contains(v_subcount,'.')):
                        pos = len(v_subcount)-v_subcount.find('.')-1 #length from the end
                        a1 = v_subcount.replace('.', '', 1)
                        num = (int(a1[:len(a1)-1])*1000)/(10**(pos-1))
                        v_subcount = num
                    else:
                        v_subcount = int(v_subcount[ :len(v_subcount)-1])*(1000)
                else:
                    v_subcount = int(float(v_subcount))
            except:
                v_subcount = 1
            v_urls = browser.find_elements_by_xpath('//div[@class="ytp-cued-thumbnail-overlay-image"]')
            for i in v_urls:
                v_url=i.get_attribute('style')[23:-3]
                break
            v_date = browser.find_element_by_xpath("//div[@id='date']").text
            if(operator.contains(v_date,'Premiered')):
                v_date = v_date[10:]
            elif(operator.contains(v_date,'Streamed live on')):
                v_date = v_date[17:]

            v_date = v_date[1:].replace(",","",1)
            v_days = number_of_days(v_date)
            url = x
            co =0
#             try:
#                 url = x
#                 html_content = urlopen(url)
#                 soup = BeautifulSoup(html_content,'lxml')
#                 v_tags = [tag["content"] for tag in soup.findAll("meta",property="og:video:tag")]
#             except:
#                 print('Error')
#             co = 0
            
#             for tl in tag_list:
#                 for t in v_tags:
#                     if operator.contains(t, tl):
#                         co+=1
#             v_tag = co/(len(tag_list))
            
            # Score formula
            try:
                v_score = (((70*v_views)/700000) + (50*(v_likes/19000)) - (20*(v_dislikes/370)) + (160*v_like_dislike) +(20*(v_views/v_days)/7000)+(30*(v_likes/v_days)/200))/31
            except:
                v_days = 1
                v_likes = 1
                v_score = (((70*v_views)/700000) + (50*(v_likes/19000)) - (20*(v_dislikes/370)) + (160*v_like_dislike)+(20*(v_views/v_days)/7000)+(30*(v_likes/v_days)/200) )/31
            
            
            print('{}:  {:.2f}'.format(url, v_score))

            
            df.loc[len(df)] = [v_link,v_title,v_url,v_views,v_likes,v_dislikes,v_subcount,v_date,v_days,v_score]
            k
            k+=1
            print(k)
        #Exception added since some YouTube ads bypass the adblock
        except Exception as e:
            #print(e)
            continue
    #df = df.sort_values(by='Score', ascending = 0)
    return df
    browser.close()

n = int(input("How many Links would you like to print? "))
df1 = search()
df = df1.nlargest(n,'Score')
print(df)

How many Links would you like to print? 5
Enter title: Ronaldo
Enter description: Goals
ronaldo goals 
['ronaldo', 'goals', '']
Scraping search results for the keyword: ronaldo goals  
https://www.youtube.com/watch?v=vUyXa1SOeXQ:  45.40
1
https://www.youtube.com/watch?v=2sJKHj2YVHc:  70.72
2
https://www.youtube.com/watch?v=2GSF9N15P2o:  16.23
3
https://www.youtube.com/watch?v=AnsBxRCRzYY:  46.08
4
https://www.youtube.com/watch?v=aWlbZw4I1jo:  27.89
5
https://www.youtube.com/watch?v=5twveLmWhvI:  64.01
6
https://www.youtube.com/watch?v=pUQErA2SxPU:  78.35
7
https://www.youtube.com/watch?v=pyu1qknWTBI:  5.37
8
https://www.youtube.com/watch?v=h84CyqHNBCE:  7.82
9
https://www.youtube.com/watch?v=de4SZdfryYE:  32.93
10
https://www.youtube.com/watch?v=sSgamVx9Rl4:  26.10
11
https://www.youtube.com/watch?v=-RiOqcefvcI:  8.81
12
https://www.youtube.com/watch?v=iK7QGGJ-cKU:  5.34
13
https://www.youtube.com/watch?v=PHYh-uyqzV4:  7.32
14
https://www.youtube.com/watch?v=WvqUCSw01Cw:  52.40
15
http

In [3]:
df1
df_likes = df1.sort_values(by='Likes',ascending=0)
df_l = df_likes.head()
list1 = []
#This part
for row in df_l.index: 
    list1.append(row)
df1
print(list1)

[5, 14, 20, 19, 34]


In [44]:
df1.sort_values(by='Score',ascending=0)

,Link,Title,URL,Views,Likes,Dislikes,Subcount,Date,Days,Score
51,https://www.youtube.com/watch?v=5hFd6zGkxLE,"THANK YOU, CRISTIANO RONALDO | Real Madrid Off...",https://i.ytimg.com/vi_webp/5hFd6zGkxLE/maxres...,31751603,1.6e+06,42000.0,5.63e+06,Jul 10 2018,664,186.106030
2,https://www.youtube.com/watch?v=pUQErA2SxPU,Lionel Messi vs Cristiano Ronaldo Best Goals Ever,https://i.ytimg.com/vi_webp/pUQErA2SxPU/maxres...,486319,17000,318.0,3.54e+06,May 3 2020,1,134.602241
1,https://www.youtube.com/watch?v=2sJKHj2YVHc,Cristiano Ronaldo ● Top 10 Unimaginable Goals ...,https://i.ytimg.com/vi_webp/2sJKHj2YVHc/maxres...,20506128,107000,6400.0,3.45e+06,Sep 1 2016,1341,70.720480
36,https://www.youtube.com/watch?v=8Us1zQkI8Ow,New Football Talent: Cristiano Ronaldo JR. (Fo...,https://i.ytimg.com/vi_webp/8Us1zQkI8Ow/maxres...,19925259,114000,7300.0,1,Oct 27 2018,555,70.357156
5,https://www.youtube.com/watch?v=5twveLmWhvI,10 Impossible Goals Scored By Lionel Messi Tha...,https://i.ytimg.com/vi_webp/kgDkGClJAwc/maxres...,19417690,249000,16000.0,441000,Nov 9 2017,907,64.007535
14,https://www.youtube.com/watch?v=dbp91G9EA8U,Cristiano Ronaldo - Taki Taki | Juventus | Ski...,https://i.ytimg.com/vi_webp/dbp91G9EA8U/maxres...,15856413,120000,9000.0,1.12e+06,Dec 19 2018,502,54.485418
15,https://www.youtube.com/watch?v=WvqUCSw01Cw,Cristiano Ronaldo UCL Goals That Shocked The W...,https://i.ytimg.com/vi_webp/WvqUCSw01Cw/maxres...,15505635,145000,11000.0,272000,Jan 15 2019,475,52.402141
32,https://www.youtube.com/watch?v=EH6p_9wXq1M,Cristiano Ronaldo - Taki Taki | Skills & Goals...,https://i.ytimg.com/vi_webp/EH6p_9wXq1M/maxres...,12884037,125000,6900.0,247000,Oct 30 2018,552,48.264550
38,https://www.youtube.com/watch?v=X3fBQSIdhyE,Cristiano Ronaldo's Dramatic Last Minute Goal!...,https://i.ytimg.com/vi_webp/F5cPdjZTeHo/maxres...,11887785,87000,5600.0,1,Sep 27 2019,220,47.691268
6,https://www.youtube.com/watch?v=AnsBxRCRzYY,Ronaldo Phenomenon Top 15 Crazy Goals \ Top 15...,https://i.ytimg.com/vi_webp/F5cPdjZTeHo/maxres...,12836804,56000,4100.0,1.5e+06,Aug 6 2018,637,46.079896


In [13]:
import plotly.graph_objects as go

colors = ['lightslategray'] * len(df1)
#This part
for i in df1.index:
    if i in list1:
        colors[i] = 'green'
    else:
        colors[i] = 'crimson'

fig = go.Figure(data=[go.Bar(
    x=[i for i in range(len(df1))],
    y=[i for i in df1.Likes],
    marker_color=colors # marker color can be a single color value or an iterable
)])
fig.update_layout(title_text='Likes per video')

In [11]:
import plotly.graph_objects as go

colors = ['lightslategray'] * len(df1)

for i in df1.index:
    if i in list1:
        colors[i] = 'green'
    else:
        colors[i] = 'crimson'

fig = go.Figure(data=[go.Bar(
    x=[i for i in range(len(df1))],
    y=[i for i in df1.Dislikes],
    marker_color=colors # marker color can be a single color value or an iterable
)])
fig.update_layout(title_text='Dislikes per video')

In [15]:
import plotly.graph_objects as go

colors = ['lightslategray'] * len(df1)

for i in df1.index:
    if i in list1:
        colors[i] = 'green'
    else:
        colors[i] = 'crimson'

fig = go.Figure(data=[go.Bar(
    x=[i for i in range(len(df1))],
    y=[i for i in df1.Views],
    marker_color=colors # marker color can be a single color value or an iterable
)])
fig.update_layout(title_text='Views per video')

In [18]:
import plotly.graph_objects as go

colors = ['lightslategray'] * len(df1)

for i in df1.index:
    if i in list1:
        colors[i] = 'green'
    else:
        colors[i] = 'crimson'

fig = go.Figure(data=[go.Bar(
    x=[i for i in range(len(df1))],
    y=[i for i in df1.Subcount],
    marker_color=colors # marker color can be a single color value or an iterable
)])
fig.update_layout(title_text='Subcount per video')

In [17]:
import plotly.graph_objects as go

colors = ['lightslategray'] * len(df1)

for i in df1.index:
    if i in list1:
        colors[i] = 'green'
    else:
        colors[i] = 'crimson'

fig = go.Figure(data=[go.Bar(
    x=[i for i in range(len(df1))],
    y=[i for i in df1.Score],
    marker_color=colors # marker color can be a single color value or an iterable
)])
fig.update_layout(title_text='Score per video')